In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Monday, August 19, 2024
Last Modified: Monday, September 15, 2025
Program Purpose: To determine the top contributing ecoregions, and to create the appropriate plots to accompany the findings. Also to conduct
the permutation analysis of all of the other ecoregions compared to the Cerrado ecoregion.
"""

In [ ]:
#Importing the appropriate libraries
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import glob
import netCDF4 as nc
import pandas as pd
import random
from heapq import nlargest
import statistics as stats
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import random
from scipy.stats import permutation_test

#Making font times
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block handles the calculation of the IAV for all of the different ecoregions

#Loading ecoregions
ecoregions = np.load("wwf_terr_ecos_360_720.npy")

# Creating a dictionary to keep all of the IAV values in
all_IAV_values = OrderedDict()

# Adding in each unique ecoregion into the dictionary by adding in all of the regions except the Oceans and lakes that are 0 and -9998
unique_regions = np.unique(ecoregions)
unique_regions = list(unique_regions)
unique_regions.remove(0)
unique_regions.remove(-9998)
print(len(unique_regions)) #Seeing how many regions to handle
# Doing the adding in with this loop:
for region in unique_regions:
    all_IAV_values[region] = OrderedDict() # Making each unique region start with an empty dict that we will populate with the IAV value for each region by model.
print(len(all_IAV_values.keys())) #Verify still 764 regions

# Converting the array into one that is across 52 (years)
ecoregions_52 = np.tile(ecoregions, (52,1,1))

# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

# Loading and procesing all of the files.
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

# Iterate over the files
for file in files:
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760
    terrestrial_flux *= area_file
    terrestrial_flux *= 1e-12
    terrestrial_flux = np.flip(terrestrial_flux, axis=1)
    dataset.close()

    # Removing extremely high values
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0

    # Removing water values
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d = data_modis[np.newaxis, :, :] # Making the data 3D
        data_modis_52 = np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    masked_array_dict = OrderedDict()
    for region in unique_regions: # Iterating through for each region and adding the masked array of that region to the dict.
        masked_array_dict[region] = np.ma.array(terrestrial_flux, mask = ecoregions_52 != region)

    yearly_flux_dict = OrderedDict()
    for region in masked_array_dict: # Gathering a list of each regions flux by year
        yearly_flux_dict[region] = masked_array_dict[region].sum(axis=(1,2)).tolist()

    
    yearly_average_flux_dict = OrderedDict()
    for region in yearly_flux_dict: # Getting the average flux for each region across the time series.
        try:
            yearly_average_flux_dict[region] = sum(yearly_flux_dict[region]) / 52
        except TypeError: #This is a nonetype issue being caught and setting the value to 0
            yearly_average_flux_dict[region] = 0

    yearly_regional_anomaly = OrderedDict()
    for region in yearly_average_flux_dict: # Adding the yearly anomalies to a list contained within the ordered dictionary.
        yearly_regional_anomaly[region] = list()
        for year in range(52):
            try:
                yearly_regional_anomaly[region].append(yearly_flux_dict[region][year] - yearly_average_flux_dict[region]) 
            except TypeError or AttributeError: #Nonetype or attribute error setting to 0
                yearly_regional_anomaly[region].append(0)


    # Now that we have all of the regional flux anomalies across all of the years, we can calculate the yearly global flux anomaly.
    yearly_global_flux_anomaly = list()
    for year in range(52): # For all 52 years summing the flux anomalies for all of the regions to get the yearly flux anomaly
        value_to_add = 0
        for region in yearly_regional_anomaly:
            value_to_add += yearly_regional_anomaly[region][year]
        yearly_global_flux_anomaly.append(value_to_add)

    # Looking for each regions IAV value now
    bottom_sum = 0
    for i in range(52): # The bottom sum is the total sum of the yearly global flux anomaly
        bottom_sum += abs(yearly_global_flux_anomaly[i])
    for region in yearly_regional_anomaly:
        top_sum = 0
        for i in range(52): # Summing for the 52 years.
            top_sum += (yearly_regional_anomaly[region][i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
        try:
            all_IAV_values[region][file_name] = (top_sum / bottom_sum * 100) # Multipy by 100 to make out of 100%
        except ZeroDivisionError: # Setting regions that are handled as NONE above to having ZERO IAV
            all_IAV_values[region][file_name] = 0
        


In [ ]:
#This block handles the conversion of the findings into an excel sheet of all of the findings
dataframe = pd.DataFrame(all_IAV_values)
dataframe.to_excel('WWF_Ecoregions_IAV.xlsx', index=True)

In [ ]:
#This block handles the gathering of the top 10 most contributing ecoregions based on the median contirbution of all 18 models

#Collecting the top median regions
median_region_IAV = OrderedDict()

# For each region in the all_IAV_values
for region in all_IAV_values:
    list_IAV = []
    # For each model in each region
    for model in all_IAV_values[region]:
        list_IAV.append(all_IAV_values[region][model])
    # Adding the median IAV value across the models to the ordered dict
    # Using the value as the key and region as the value, so can sort keys below to easily get the top 10 regions
    median_region_IAV[np.median(list_IAV)] = region

# Gathering the top 10 regions for IAV based on the average of the models
list_median_regional_IAV = []
for key in median_region_IAV.keys():
    list_median_regional_IAV.append(float(key))

#Sorting the list from biggest to smallest and taking the top values
list_median_regional_IAV.sort(reverse=True)
top_10_regions_median = OrderedDict()
for i in range(10):
    top_10_regions_median[median_region_IAV[list_median_regional_IAV[i]]] = list_median_regional_IAV[i]

#Printing the results
#The names of the regions can be found by crossreferenceing the ecoregions excel workbook
print(top_10_regions_median)
print(top_10_regions_median.keys()) #These are the top 10 median regions

#Naming the ecoregions based on the excel workbook
cerrado_IAV = OrderedDict()
east_siberian_IAV = OrderedDict()
miombo_IAV = OrderedDict()
caatinga_IAV = OrderedDict()
mopane_IAV = OrderedDict()
south_miombo_IAV = OrderedDict()
guianan_moist_IAV = OrderedDict()
west_sudan_IAV = OrderedDict()
scandanavian_IAV = OrderedDict()
amazon_moist_IAV = OrderedDict()

#Adding the values to the resepective dicts
for region in all_IAV_values:
    if int(region) in (60704, 80601, 30704, 61304, 30725, 30719, 60173, 60125, 30722, 80608):
    # For each model in each region
        for model in all_IAV_values[region]:
            if region == 60704:
                cerrado_IAV[model] = all_IAV_values[region][model]

            if region == 80601:
                east_siberian_IAV[model] = all_IAV_values[region][model]

            if region ==  30704:
                miombo_IAV [model] = all_IAV_values[region][model]
            
            if region ==  61304:
                caatinga_IAV [model] = all_IAV_values[region][model]

            if region ==  30725:
                mopane_IAV [model] = all_IAV_values[region][model]

            if region ==  30719:
                south_miombo_IAV [model] = all_IAV_values[region][model]

            if region ==  60173:
                amazon_moist_IAV [model] = all_IAV_values[region][model]

            if region ==  60125:
                guianan_moist_IAV [model] = all_IAV_values[region][model]

            if region == 30722:
                west_sudan_IAV [model] = all_IAV_values[region][model]

            if region ==  80608:
                scandanavian_IAV [model] = all_IAV_values[region][model]


In [ ]:
#Collecting the top median regions
mean_region_IAV = OrderedDict()

# For each region in the all_IAV_values
for region in all_IAV_values:
    list_IAV = []
    # For each model in each region
    for model in all_IAV_values[region]:
        list_IAV.append(all_IAV_values[region][model])
    # Adding the median IAV value across the models to the ordered dict
    # Using the value as the key and region as the value, so can sort keys below to easily get the top 10 regions
    mean_region_IAV[np.mean(list_IAV)] = region

#Getting the total sum of the median values
sum = 0
for value in mean_region_IAV:
    sum += value
print(sum)

In [ ]:
#This block handles the generation of the boxplot for the top 10 absolute most contributing ecoregions

#Compiling the top 10 absolute contributing ecoregions together
compiled_IAV_data = [list(cerrado_IAV.values()) , list(east_siberian_IAV.values()), list(miombo_IAV.values()), list(caatinga_IAV.values()), list(mopane_IAV.values()), list(south_miombo_IAV.values()), list(amazon_moist_IAV.values()), list(guianan_moist_IAV.values()), list(west_sudan_IAV.values()), list(scandanavian_IAV.values())]

# Plotting the boxplots
fig, ax = plt.subplots(figsize=(40, 20))

boxplots = ax.boxplot(compiled_IAV_data, widths=0.8, patch_artist=True, showmeans=True,  meanprops={"marker": "D", "markerfacecolor": "grey", "markeredgecolor": "black", "markersize": 22}, 
    flierprops=dict(marker="o", markersize=22, linestyle="none", markeredgewidth=8),  whiskerprops=dict(linewidth=8),  capprops=dict(linewidth=8),  boxprops=dict(linewidth=8),
    medianprops=dict(linewidth=8))

ax.set_xticklabels(['Cerrado', 'East Siberian\nTaiga', 'Central Zambezian\nMiombo Woodlands', 
     'Caatinga', 'Zambezian and\nMopane Woodlands', 'Southern\n Miombo Woodlands', "Uatuma-Trombetas\n Moist Forests", 
     'Guianan\nMoist Forests', 'West Sudanian\nSavanna', 'Scandinavian\nAnd\nRussian Taiga',
     ], rotation=60, fontsize=55, fontweight='bold')

#Custom colors for colorblind friendly
colors = ["#db6d00", "#ffdf4d", "#8f4e00", "#006ddb", "#000000", "#ff6db6", "#22cf22", "#009999", "#676767", "#490092"]

#Modifying the colors across the boxes
boxes = boxplots['boxes']
whiskers = boxplots['whiskers']
caps = boxplots['caps']
fliers = boxplots['fliers']
medians = boxplots['medians']

i = 0
for box in boxes:
    box.set_facecolor('white')
    box.set_edgecolor(colors[i])
    box.set
    i+=1

i = 0
for median in medians:
    median.set_color(colors[i])
    i+=1


i = 0
j=0
for whisker in whiskers:
    whisker.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

i = 0
j=0
for cap in caps:
    cap.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

fliers[2].set(markerfacecolor = 'white', markeredgecolor=colors[2])
fliers[3].set(markerfacecolor = 'white', markeredgecolor=colors[3])
fliers[4].set(markerfacecolor = 'white', markeredgecolor=colors[4])
fliers[7].set(markerfacecolor = 'white', markeredgecolor=colors[7])
ax.set_xlabel("Ecoregion", fontsize=55, fontweight='bold')
ax.set_ylabel("IAV Contribution (%)", fontsize=55, fontweight='bold')
ax.tick_params(axis='y', labelsize=70, width=20)
ax.tick_params(axis='x', width=20)
plt.ylim(-2, 8.5)
#Adding title
plt.title("(b)", fontsize=70, loc='left', fontweight='bold')

#Adding the lines denoting significance
plt.text(1, max(cerrado_IAV.values())+0.5, 'A', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(2, max(east_siberian_IAV.values())+0.5, 'ABC', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(3, max(miombo_IAV.values())+0.5, 'B', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(4, max(caatinga_IAV.values())+0.5, 'BC', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(5, max(mopane_IAV.values())+0.5, 'BC', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(6, max(south_miombo_IAV.values())+0.5, 'C', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(7, max(amazon_moist_IAV.values())+0.5, 'C', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(8, max(guianan_moist_IAV.values())+0.5, 'C', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(9, max(west_sudan_IAV.values())+0.5, 'C', dict(size=55, weight='bold'), horizontalalignment='center')
plt.text(10, max(scandanavian_IAV.values())+0.5, 'C', dict(size=55, weight='bold'), horizontalalignment='center')

plt.show()


In [ ]:
#This blcok handles the creation of the barplot of all of the top contributing ecoregions by model

#Figure size and spacing
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 5.5

#Colors same as above being colorblind friendly
colors = ["#db6d00", "#ffdf4d", "#8f4e00", "#006ddb", "#000000", "#ff6db6", "#22cf22", "#009999", "#676767", "#490092"]

#Creating the bar plot
plt.bar(pos - 9*width/2, cerrado_IAV.values(), width=width, label='Cerrado', color=colors[0])
plt.bar(pos- 7*width/2, east_siberian_IAV.values(), width=width, label='East Siberian Taiga', color=colors[1])
plt.bar(pos - 5*width/2, miombo_IAV.values(), width=width, label='Central Zambezian Miombo Woodlands', color=colors[2])
plt.bar(pos - 3*width/2,  caatinga_IAV.values(), width=width, label='Zambezian and Mopane Woodlands', color=colors[3])
plt.bar(pos - width/2, mopane_IAV.values(), width=width, label='Caatinga', color=colors[4])
plt.bar(pos + width/2, south_miombo_IAV.values(), width=width, label='Southern Miombo Woodlands', color=colors[5])
plt.bar(pos + 3*width/2, amazon_moist_IAV.values(), width=width, label='Uatuma-Trombetas Moist Forest', color=colors[6])
plt.bar(pos + 5*width/2, guianan_moist_IAV.values(), width=width, label='Guianan Moist Forests', color=colors[7])
plt.bar(pos + 7*width/2, west_sudan_IAV.values(), width=width, label='West Sudanian Savanna', color=colors[8])
plt.bar(pos + 9*width/2, scandanavian_IAV.values(), width=width, label="Scandinavian and Russian Taiga", color=colors[9])

#Making proper model name labels
new_labels = []
for key in cerrado_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)

#Adding labels
plt.xlabel('Model',fontsize=25)
plt.ylabel(' IAV Contribution (%)',fontsize=25)
plt.yticks(fontsize=25)
#plt.title("Comparison of models IAV for Top 10 Mean Regions of Interest",fontsize=20)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.ylim(-1.5,8.8)
plt.legend(fontsize=20, ncol=3, loc="upper center")

#Showing plot
plt.show()

In [ ]:
#This block handles the gathering of the stats required to report and to execute the permutation testing

#Getting the median and IQR values
print(f"\nCerrado median IAV: {round(stats.median(cerrado_IAV.values()), 2)}")
print(f"Cerrado 25: {round(np.percentile(np.array(list(cerrado_IAV.values())), 25), 2)}")
print(f"Cerrado 75: {round(np.percentile(np.array(list(cerrado_IAV.values())), 75), 2)}")

print(f"\nEast Siberian Taiga median IAV: {round(stats.median(east_siberian_IAV.values()), 2)}")
print(f"East Siberian 25: {round(np.percentile(np.array(list(east_siberian_IAV.values())), 25), 2)}")
print(f"East Siberian 75: {round(np.percentile(np.array(list(east_siberian_IAV.values())), 75), 2)}")

print(f"\nCentral Zambezian Miombo Woodlands median IAV: {round(stats.median(miombo_IAV.values()), 2)}")
print(f"Central Zambezian Miombo Woodlands 25: {round(np.percentile(np.array(list(miombo_IAV.values())), 25), 2)}")
print(f"Central Zambezian Miombo Woodlands 75: {round(np.percentile(np.array(list(miombo_IAV.values())), 75), 2)}")

print(f"\nCaatinga median IAV: {round(stats.median(caatinga_IAV.values()), 2)}")
print(f"Caatinga 25: {round(np.percentile(np.array(list(caatinga_IAV.values())), 25), 2)}")
print(f"Caatinga 75: {round(np.percentile(np.array(list(caatinga_IAV.values())), 75), 2)}")

print(f"\nZambezian and Mopane Woodlands median IAV: {round(stats.median(mopane_IAV.values()), 2)}")
print(f"Zambezian and Mopane 25: {round(np.percentile(np.array(list(mopane_IAV.values())), 25), 2)}")
print(f"Zambezian and Mopane 75: {round(np.percentile(np.array(list(mopane_IAV.values())), 75), 2)}")

print(f"\nSouthern Miombo Woodlands median IAV: {round(stats.median(south_miombo_IAV.values()), 2)}")
print(f"Southern Miombo Woodlands 25: {round(np.percentile(np.array(list(south_miombo_IAV.values())), 25), 2)}")
print(f"Southern Miombo Woodlands 75: {round(np.percentile(np.array(list(south_miombo_IAV.values())), 75), 2)}")

print(f"\nUatuma-Trombetas Moist Forests median IAV: {round(stats.median(amazon_moist_IAV.values()), 2)}")
print(f"Uatuma-Trombetas Moist Forests 25: {round(np.percentile(np.array(list(amazon_moist_IAV.values())), 25), 2)}")
print(f"Uatuma-Trombetas Moist Forests 75: {round(np.percentile(np.array(list(amazon_moist_IAV.values())), 75), 2)}")

print(f"\nGuianan Moist Forests median IAV: {round(stats.median(guianan_moist_IAV.values()), 2)}")
print(f"Guianan Moist Forests 25: {round(np.percentile(np.array(list(guianan_moist_IAV.values())), 25), 2)}")
print(f"Guianan Moist Forests 75: {round(np.percentile(np.array(list(guianan_moist_IAV.values())), 75), 2)}")

print(f"\nWest Sudanese Savana median IAV: {round(stats.median(west_sudan_IAV.values()), 2)}")
print(f"West Sudanian Savana 25: {round(np.percentile(np.array(list(west_sudan_IAV.values())), 25), 2)}")
print(f"West Sudanian Savana 75: {round(np.percentile(np.array(list(west_sudan_IAV.values())), 75), 2)}")

print(f"\nScandinavian and Russian Taiga median IAV: {round(stats.median(scandanavian_IAV.values()), 2)}")
print(f"Scandinavian and Russian Taiga 25: {round(np.percentile(np.array(list(scandanavian_IAV.values())), 25), 2)}")
print(f"Scandinavian and Russian Taiga 75: {round(np.percentile(np.array(list(scandanavian_IAV.values())), 75), 2)}")

# Getting all of the individual values from each model for all of the ecoregions so that the permutation testing can be done
cerrado_values = []
for key in cerrado_IAV:
    cerrado_values.append(round(float(cerrado_IAV[key]),4))
print(f"Cerrado = {cerrado_values}")

east_siberian_values = []
for key in east_siberian_IAV:
    east_siberian_values.append(round(float(east_siberian_IAV[key]),4))
print(f"East_Siberian = {east_siberian_values}")

miombo_values = []
for key in miombo_IAV:
    miombo_values.append(round(float(miombo_IAV[key]),4))
print(f"Central_Zambezian_Miombo = {miombo_values}")

caatinga_values = []
for key in caatinga_IAV:
    caatinga_values.append(round(float(caatinga_IAV[key]),4))
print(f"Caatinga = {caatinga_values}")

mopane_values = []
for key in mopane_IAV:
    mopane_values.append(round(float(mopane_IAV[key]),4))
print(f"Zambezian_Mopane = {mopane_values}")

south_miombo_values = []
for key in south_miombo_IAV:
    south_miombo_values.append(round(float(south_miombo_IAV[key]),4))
print(f"South_Miombo = {south_miombo_values}")

amazon_moist_values = []
for key in amazon_moist_IAV:
    amazon_moist_values.append(round(float(amazon_moist_IAV[key]),4))
print(f"Amazon_Moist = {amazon_moist_values}")

guianan_moist_values = []
for key in guianan_moist_IAV:
    guianan_moist_values.append(round(float(guianan_moist_IAV[key]),4))
print(f"Guianan_Moist = {guianan_moist_values}")

west_sudan_values = []
for key in west_sudan_IAV:
    west_sudan_values.append(round(float(west_sudan_IAV[key]),4))
print(f"West_Sudan = {west_sudan_values}")

scandanavian_values = []
for key in scandanavian_IAV:
    scandanavian_values.append(round(float(scandanavian_IAV[key]),4))
print(f"Scandinavian_Russian_Taiga = {scandanavian_values}")



In [ ]:
#This block handles the gathering of the top 10 most contributing ecoregions based on the area-weighted IAV contribution

# Creating a dictionary to keep all of the IAV in.
area_weighted_IAV_values = OrderedDict()
for region in unique_regions:
    area_weighted_IAV_values[region] = OrderedDict()

#Getting the area that all of the different ecoregions take up after accounting for the water masking using the MODIS data
modis_data = np.load("MODIS_Map.npy")
land_global_area = area_file[modis_data != 0]
global_surface_area = np.sum(land_global_area)

#Getting the area of all of the ecoregions
dict_all_ecoregion_area_percent = {} #the regions are the key, and the value is the area
for region in unique_regions:
    temp_map = np.zeros((360,720)) #temp map
    temp_map[ecoregions == region] = 1 #setting that regions cells == 1
    area_map = temp_map*area_file
    total_area = np.sum(area_map)
    dict_all_ecoregion_area_percent[region] = total_area / global_surface_area * 100

total_area = 0
#Verifying the global areas of the ecoregions
for region in dict_all_ecoregion_area_percent:
    total_area += dict_all_ecoregion_area_percent[region]
print(f"The total area == {total_area}... Greater than 100 because of rounding")

#Calculating an area-weighted IAV contribution for all of the regions
for region in area_weighted_IAV_values:
    #Accessing the absolute IAV values for each model and calculating the area-weighted contribution
    for model in all_IAV_values[region]:
        #area_weighted IAV contribution = IAV contribution % / biome area %
        area_weighted_IAV_values[region][model] = all_IAV_values[region][model] / dict_all_ecoregion_area_percent[region] 

#Getting the top 10 greatest contriuting median area-weighted IAV contributors
all_median_IAV_contributions = {}
for region in area_weighted_IAV_values:
    temp_list = []
    for model in area_weighted_IAV_values[region]:
        temp_list.append(area_weighted_IAV_values[region][model])
    median_contribution = np.median(temp_list)
    all_median_IAV_contributions[region] = median_contribution

#The names of the regions can be found by crossreferenceing the ecoregions excel workbook
print(f"The 10 largest area-weighted IAV contributions are: {nlargest(10, all_median_IAV_contributions, key=all_median_IAV_contributions.get)}")

#Printing the areas of the regions identified to be the top absolute and area-weighted contributing ecoregions
#Absolute top region areas
print(f"\nArea Cerrado:{round(dict_all_ecoregion_area_percent[60704], 2)}")
print(f"Area East Siberian:{round(dict_all_ecoregion_area_percent[80601], 2)}")
print(f"Area Central Zambezian Miombo:{round(dict_all_ecoregion_area_percent[30704], 2)}")
print(f"Area Caatinga:{round(dict_all_ecoregion_area_percent[61304], 2)}")
print(f"Area Zambezian and Mopane Woodlands:{round(dict_all_ecoregion_area_percent[30725], 2)}")
print(f"Area Southern Miombo Woodlands:{round(dict_all_ecoregion_area_percent[30719], 2)}")
print(f"Area Uatuma-Trombetas:{round(dict_all_ecoregion_area_percent[60173], 2)}")
print(f"Area Guianan Moist:{round(dict_all_ecoregion_area_percent[60125], 2)}")
print(f"Area West Sudanian:{round(dict_all_ecoregion_area_percent[30722], 2)}")
print(f"Area Scandinavian and Russian:{round(dict_all_ecoregion_area_percent[80608], 2)}")
#Area-weighted top region areas
print(f"\nArea Western Zambezian Grasslands:{round(dict_all_ecoregion_area_percent[30724], 2)}")
print(f"Area Eastern Zimbabwe Montane:{round(dict_all_ecoregion_area_percent[31006], 2)}")
print(f"Area Zambezian Baikiaea:{round(dict_all_ecoregion_area_percent[30726], 2)}")
print(f"Area Southern Miombo:{round(dict_all_ecoregion_area_percent[30719], 2)}")
print(f"Area Chao Phraya:{round(dict_all_ecoregion_area_percent[40108], 2)}")
print(f"Area Guianan Savanna:{round(dict_all_ecoregion_area_percent[60707], 2)}")
print(f"Area Maranhao:{round(dict_all_ecoregion_area_percent[60139], 2)}")
print(f"Area Zambezian Coastal Flooded:{round(dict_all_ecoregion_area_percent[30906], 2)}")
print(f"Area Pernambuco:{round(dict_all_ecoregion_area_percent[60152], 2)}")
print(f"Area Zambezian and Mopane:{round(dict_all_ecoregion_area_percent[30725], 2)}")



In [ ]:
#This block handles the making the boxplot of these top 10 area-weighted regions and the contribution that they make

#Setting up the dicts to populate
western_zambezian_grasslands = OrderedDict()
eastern_zimbabwe_montane = OrderedDict()
zambezian_baikiaea = OrderedDict()
southern_miombo = OrderedDict()
chao_phraya = OrderedDict()
guianan_savanna = OrderedDict()
maranhao_babacu = OrderedDict()
zambezian_coastal = OrderedDict()
pernambuco_interior = OrderedDict()
zambeizan_and_mopane = OrderedDict()

#Placing the values into the appropriate dict
for model in area_weighted_IAV_values[30724]:
    western_zambezian_grasslands[model] = area_weighted_IAV_values[30724][model]

for model in area_weighted_IAV_values[31006]:
    eastern_zimbabwe_montane[model] = area_weighted_IAV_values[31006][model]

for model in area_weighted_IAV_values[30726]:
    zambezian_baikiaea[model] = area_weighted_IAV_values[30726][model]

for model in area_weighted_IAV_values[30719]:
    southern_miombo[model] = area_weighted_IAV_values[30719][model]

for model in area_weighted_IAV_values[40108]:
    chao_phraya[model] = area_weighted_IAV_values[40108][model]

for model in area_weighted_IAV_values[60707]:
    guianan_savanna[model] = area_weighted_IAV_values[60707][model]

for model in area_weighted_IAV_values[60139]:
    maranhao_babacu[model] = area_weighted_IAV_values[60139][model]

for model in area_weighted_IAV_values[30906]:
    zambezian_coastal[model] = area_weighted_IAV_values[30906][model]

for model in area_weighted_IAV_values[60152]:
    pernambuco_interior[model] = area_weighted_IAV_values[60152][model]

for model in area_weighted_IAV_values[30725]:
    zambeizan_and_mopane[model] = area_weighted_IAV_values[30725][model]


#Putting all of the values into one list, ordered from the greatest contributor to the least contributor
area_weighted_IAV_data_boxes = [list(western_zambezian_grasslands.values()), list(eastern_zimbabwe_montane.values()), list(zambezian_baikiaea.values()), list(southern_miombo.values()), list(chao_phraya.values()), list(guianan_savanna.values()), list(maranhao_babacu.values()), list(zambezian_coastal.values()), list(pernambuco_interior.values()),  list(zambeizan_and_mopane.values())]

# Plotting the boxplots
fig, ax = plt.subplots(figsize=(40, 20))

boxplots = ax.boxplot(area_weighted_IAV_data_boxes, widths=0.8, patch_artist=True, showmeans=True,  meanprops={"marker": "D", "markerfacecolor": "grey", "markeredgecolor": "black", "markersize": 22}, 
    flierprops=dict(marker="o", markersize=22, linestyle="none", markeredgewidth=8),  whiskerprops=dict(linewidth=8),  capprops=dict(linewidth=8),  boxprops=dict(linewidth=8),
    medianprops=dict(linewidth=8))

ax.set_xticklabels(['Western\nZambezian\nGrasslands', 'Eastern Zimbabwe\nMontane\nForest-Grassland\nMosaic', 'Zambezian\nBaikiaea Woodlands', 
     'Southern Miombo\nWoodlands', 'Chao Phraya\nLowland Moist\nDeciduous Forests', 'Guianan\nSavanna', 
     'Maranhão Babaçu\nForests', 'Zambezian\nCoastal Flooded\nSavanna', 'Pernambuco\nInterior Forests',
     "Zambezian\nAnd\nMopane Woodlands"], 
    rotation=60, fontsize=55, fontweight='bold')

#Custom colors
colors = ["#db6d00", "#ffdf4d", "#8f4e00", "#006ddb", "#000000", "#ff6db6", "#22cf22", "#009999", "#676767", "#490092"]

#Modifying the colors across the boxes
boxes = boxplots['boxes']
whiskers = boxplots['whiskers']
caps = boxplots['caps']
fliers = boxplots['fliers']
medians = boxplots['medians']

i = 0
for box in boxes:
    box.set_facecolor('white')
    box.set_edgecolor(colors[i])
    box.set
    i+=1

i = 0
for median in medians:
    median.set_color(colors[i])
    i+=1


i = 0
j=0
for whisker in whiskers:
    whisker.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

i = 0
j=0
for cap in caps:
    cap.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

fliers[0].set(markerfacecolor = 'white', markeredgecolor=colors[0])
fliers[1].set(markerfacecolor = 'white', markeredgecolor=colors[1])
fliers[5].set(markerfacecolor = 'white', markeredgecolor=colors[5])
fliers[9].set(markerfacecolor = 'white', markeredgecolor=colors[9])
ax.set_xlabel("Ecoregion", fontsize=55, fontweight='bold')
ax.set_ylabel("Area-Weighted IAV Contribution (%)", fontsize=55, fontweight='bold')

ax.tick_params(axis='y', labelsize=70, width=20)
ax.tick_params(axis='x', width=20)

#Adding title
plt.title("(b)", fontsize=70, loc='left', fontweight='bold')

plt.show()

#Also printing out the values which are in each region to conduct the permutation test between the different ecoregions
print(f"western_zambezian_grasslands: {western_zambezian_grasslands.values()}")
print(f"eastern_zimbabwe_montane: {eastern_zimbabwe_montane.values()}")
print(f"zambezian_baikiaea: {zambezian_baikiaea.values()}")
print(f"southern_miombo: {southern_miombo.values()}")
print(f"chao_phraya: {chao_phraya.values()}")
print(f"guianan_savanna: {guianan_savanna.values()}")
print(f"maranhao_babacu: {maranhao_babacu.values()}")
print(f"zambezian_coastal: {zambezian_coastal.values()}")
print(f"pernambuco_interior: {pernambuco_interior.values()}")
print(f"zambeizan_and_mopane: {zambeizan_and_mopane.values()}")


In [ ]:
#This block handles the creation of the barplot of all of the top area-weighted contributing ecoregions by model

#Figure size and spacing
plt.subplots(figsize=(24, 10))
width = 0.5
pos = np.arange(18) * 5.5

#Colors same as above
colors = ["#db6d00", "#ffdf4d", "#8f4e00", "#006ddb", "#000000", "#ff6db6", "#22cf22", "#009999", "#676767", "#490092"]

#Creating the bars
plt.bar(pos - 9*width/2, western_zambezian_grasslands.values(), width=width, label='Western\nZambezian\nGrasslands', color=colors[0])
plt.bar(pos - 7*width/2, eastern_zimbabwe_montane.values(), width=width, label='Eastern Zimbabwe\nMontane\nForest-Grassland\nMosaic', color=colors[1])
plt.bar(pos - 5*width/2, zambezian_baikiaea .values(), width=width, label='Zambezian\nBaikiaea Woodlands', color=colors[2])
plt.bar(pos - 3*width/2, southern_miombo.values(), width=width, label='Southern\nMiombo\nWoodlands', color=colors[3])
plt.bar(pos - width/2, chao_phraya.values(), width=width, label='Chao Phraya\nLowland Moist\nDeciduous Forests', color=colors[4])
plt.bar(pos + width/2, guianan_savanna.values(), width=width, label='Guianan\nSavanna', color=colors[5])
plt.bar(pos + 3*width/2, maranhao_babacu.values(), width=width, label='Maranhão Babaçu\nForests', color=colors[6])
plt.bar(pos + 5*width/2, zambezian_coastal.values(), width=width, label='Zambezian\nCoastal Flooded\nSavanna', color=colors[7])
plt.bar(pos + 7*width/2, pernambuco_interior.values(), width=width, label='Pernambuco\nInterior Forests', color=colors[8])
plt.bar(pos + 9*width/2, zambeizan_and_mopane.values(), width=width, label="Zambezian\nAnd\nMopane Woodlands", color=colors[9])

#Making proper model name labels
new_labels = []
for key in cerrado_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)

# Add labels, legend, and title
plt.xlabel('Model',fontsize=25)
plt.ylabel('Area-Weighted IAV Contribution (%)',fontsize=25)
plt.yticks(fontsize=25)
#plt.title("Comparison of models IAV for Top 10 Mean Regions of Interest",fontsize=20)
plt.xticks(pos, new_labels, fontsize=20, rotation=45)
plt.legend(fontsize=16, ncol=3, loc="upper left", framealpha=0.5)


plt.show()

In [ ]:
#This block is to report the median and IQR values for the top area-weighted contibuting regions

#Median contributions
print(f"Western Zambezian median IAV: {round(stats.median(western_zambezian_grasslands.values()), 2)}")

print(f"Eastern Zimbabwe montane IAV: {round(stats.median(eastern_zimbabwe_montane.values()), 2)}")

print(f"Zambezian Baikiaea median IAV: {round(stats.median(zambezian_baikiaea.values()), 2)}")

print(f"Southern Miombo median IAV: {round(stats.median(southern_miombo.values()), 2)}")

print(f"Chao Phraya median IAV: {round(stats.median(chao_phraya.values()), 2)}")

print(f"Guianan Savanna median IAV: {round(stats.median(guianan_savanna.values()), 2)}")

print(f"Maranhao Babacu median IAV: {round(stats.median(maranhao_babacu.values()), 2)}")

print(f"Zambezian Coastal median IAV: {round(stats.median(zambezian_coastal.values()), 2)}")

print(f"Pernambuco Interior median IAV: {round(stats.median(pernambuco_interior.values()), 2)}")

print(f"Zambezian Mopane median IAV: {round(stats.median(zambeizan_and_mopane.values()), 2)}")

#IQR values
print("\n BREAK..............\n")
print(f"Western Zambezian 25: {round(np.percentile(np.array(list(western_zambezian_grasslands.values())), 25), 2)}")
print(f"Western Zambezian 75: {round(np.percentile(np.array(list(western_zambezian_grasslands.values())), 75), 2)}")

print(f"Eastern Zimbabwe montane 25: {round(np.percentile(np.array(list(eastern_zimbabwe_montane.values())), 25), 2)}")
print(f"Eastern Zimbabwe montane 75: {round(np.percentile(np.array(list(eastern_zimbabwe_montane.values())), 75), 2)}")

print(f"Zambezian Baikiaea 25: {round(np.percentile(np.array(list(zambezian_baikiaea.values())), 25), 2)}")
print(f"Zambezian Baikiaea 75: {round(np.percentile(np.array(list(zambezian_baikiaea.values())), 75), 2)}")

print(f"Southern Miombo  25: {round(np.percentile(np.array(list(southern_miombo.values())), 25), 2)}")
print(f"Southern Miombo  75: {round(np.percentile(np.array(list(southern_miombo.values())), 75), 2)}")

print(f"Chao Phraya 25: {round(np.percentile(np.array(list(chao_phraya.values())), 25), 2)}")
print(f"Chao Phraya 75: {round(np.percentile(np.array(list(chao_phraya.values())), 75), 2)}")

print(f"Guianan Savanna  25: {round(np.percentile(np.array(list(guianan_savanna.values())), 25), 2)}")
print(f"Guianan Savanna  75: {round(np.percentile(np.array(list(guianan_savanna.values())), 75), 2)}")

print(f"Maranhao Babacu  25: {round(np.percentile(np.array(list(maranhao_babacu.values())), 25), 2)}")
print(f"Maranhao Babacu  75: {round(np.percentile(np.array(list(maranhao_babacu.values())), 75), 2)}")

print(f"Zambezian Coastal 25: {round(np.percentile(np.array(list(zambezian_coastal.values())), 25), 2)}")
print(f"Zambezian Coastal 75: {round(np.percentile(np.array(list(zambezian_coastal.values())), 75), 2)}")

print(f"Pernambuco Interior 25: {round(np.percentile(np.array(list(pernambuco_interior.values())), 25), 2)}")
print(f"Pernambuco Interior 75: {round(np.percentile(np.array(list(pernambuco_interior.values())), 75), 2)}")

print(f"Zambezian Mopane 25: {round(np.percentile(np.array(list(zambeizan_and_mopane.values())), 25), 2)}")
print(f"Zambezian Mopane 75: {round(np.percentile(np.array(list(zambeizan_and_mopane.values())), 75), 2)}")

In [ ]:
#This block is concerend with creating the map of all of the top contributing ecoregions
ecoregions = np.load("wwf_terr_ecos_360_720.npy")
#Setting up the different regions
ecoregion_absolute_top = ecoregions.copy()
#Setting the areas to zero if not a desired ecoregion
ecoregion_absolute_top[(ecoregions != 60704.0) & (ecoregions != 80601.0) & (ecoregions != 30704.0) & (ecoregions != 61304.0) & 
                       (ecoregions != 30725.0) & (ecoregions != 30719.0) & (ecoregions != 60173.0) & (ecoregions != 60125.0) & 
                       (ecoregions != 30722.0) & (ecoregions != 80608)] = 0
# Changing the number of the ecoregions, such that they are each one larger than the next.
desired_regions = [60704.0, 80601.0, 30704.0, 61304.0, 30725.0, 30719.0, 60173.0, 60125.0, 30722.0, 80608.0]
for i in range(10): # 10 unique regions and then one region of water and other land
    ecoregion_absolute_top[ecoregion_absolute_top == desired_regions[i]] = i+1

# Setting up the map projection
fig = plt.figure(figsize=(40, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plotting the global map
im = ax.imshow(ecoregion_absolute_top, cmap=ListedColormap([ 'white',"#db6d00", "#ffdf4d","#8f4e00","#006ddb","#000000","#ff6db6", "#22cf22",   "#009999", "#676767", "#490092",]),
               extent=[-180, 180, -90, 90], origin='upper', transform=ccrs.PlateCarree(), interpolation='none')
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

# Adding legend
Cerado_Bar = mpatches.Patch(facecolor="#db6d00", label='Cerrado', edgecolor='black')
East_Siberian_Bar = mpatches.Patch(facecolor="#ffdf4d", label='East Siberian Taiga', edgecolor='black')
Central_Zambezian_Bar = mpatches.Patch(facecolor="#8f4e00", label='Central Zambezian\nMiombo Woodlands', edgecolor='black')
Caatinga_Bar = mpatches.Patch(facecolor="#006ddb", label='Caatinga', edgecolor='black')
Zambezian_Mopane_Bar = mpatches.Patch(facecolor="#000000", label='Zambezian and\nMopane Woodlands', edgecolor='black')
South_Miombo_Bar = mpatches.Patch(facecolor="#ff6db6", label='Southern\n Miombo Woodlands', edgecolor='black')
Uatuma_Bar  = mpatches.Patch(facecolor="#22cf22", label='Uatuma-Trombetas\n Moist Forests', edgecolor='black')
Guianan_Moist_Bar = mpatches.Patch(facecolor="#009999", label='Guianan Moist Forests', edgecolor='black')
West_Sudanian_Bar = mpatches.Patch(facecolor="#676767", label='West Sudanian\nSavanna', edgecolor='black')
Scandinavian_Bar = mpatches.Patch(facecolor="#490092", label='Scandinavian\nAnd Russian Taiga', edgecolor='black')
#Adding the legend
plt.legend(handles=[Cerado_Bar, East_Siberian_Bar, Central_Zambezian_Bar, Caatinga_Bar, Zambezian_Mopane_Bar, South_Miombo_Bar, Uatuma_Bar, 
                    Guianan_Moist_Bar, West_Sudanian_Bar, Scandinavian_Bar], loc='upper left', fontsize=43)

#Title
plt.title("(a)", fontsize=70, loc='left', fontweight='bold')

plt.show()

In [ ]:
#This block is concerend with creating the map of the top contributing area-weighted ecoregions

#Setting up the different regions
ecoregion_area_weighted_top = ecoregions.copy()
#Setting the areas to zero if not a desired ecoregion
ecoregion_area_weighted_top[(ecoregions != 30724) & (ecoregions != 31006) & (ecoregions != 30726) & (ecoregions != 30719) & 
                       (ecoregions != 40108) & (ecoregions != 60707) & (ecoregions != 60139) & (ecoregions != 30906) & 
                       (ecoregions != 60152) & (ecoregions != 30725)] = 0
# Changing the number of the ecoregions, such that they are each one larger than the next.
desired_regions = [30724, 31006, 30726, 30719, 40108, 60707, 60139, 30906, 60152, 30725]
for i in range(10): # 10 unique regions and then one region of water and other land
    ecoregion_area_weighted_top[ecoregion_area_weighted_top == desired_regions[i]] = i+1

# Setting up the map projection
fig = plt.figure(figsize=(40, 20))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plotting the global map
im = ax.imshow(ecoregion_area_weighted_top, cmap=ListedColormap(['white', "#db6d00", "#ffdf4d", "#8f4e00", "#006ddb", "#000000", "#ff6db6", "#22cf22", "#009999", "#676767", "#490092"]),
               extent=[-180, 180, -90, 90], origin='upper', transform=ccrs.PlateCarree(), interpolation='none')
ax.set_extent([-90, 110, -30, 25], crs=ccrs.PlateCarree()) #Setting the extent of the map such that the regions of the ecoregions can actually be seen


# Add legend
western_zambezian_grasslands_Bar = mpatches.Patch(facecolor="#db6d00", label='Western Zambezian\nGrasslands', edgecolor='black')
eastern_zimbabwe_montane_Bar = mpatches.Patch(facecolor="#ffdf4d", label='Eastern Zimbabwe Montane\nForest-Grassland Mosaic', edgecolor='black')
zambezian_baikiaea_Bar = mpatches.Patch(facecolor="#8f4e00", label='Zambezian\nBaikiaea Woodlands', edgecolor='black')
southern_miombo_Bar = mpatches.Patch(facecolor="#006ddb", label='Southern Miombo\nWoodlands', edgecolor='black')
chao_phraya_Bar= mpatches.Patch(facecolor="#000000", label='Chao Phraya\nLowland Moist\nDeciduous Forests', edgecolor='black')
guianan_savanna_Bar = mpatches.Patch(facecolor="#ff6db6", label='Guianan\nSavanna', edgecolor='black')
maranhao_babacu_Bar= mpatches.Patch(facecolor="#22cf22", label='Maranhão Babaçu\nForests', edgecolor='black')
zambezian_coastal_Bar= mpatches.Patch(facecolor="#009999", label='Zambezian Coastal\nFlooded Savanna', edgecolor='black')
pernambuco_interior_Bar = mpatches.Patch(facecolor="#676767", label='Pernambuco\nInterior Forests', edgecolor='black')
zambeizan_and_mopane_Bar = mpatches.Patch(facecolor="#490092", label='Zambezian And\nMopane Woodlands', edgecolor='black')
#Adding the legend
#plt.legend(handles=[western_zambezian_grasslands_Bar, eastern_zimbabwe_montane_Bar, zambezian_baikiaea_Bar, southern_miombo_Bar, chao_phraya_Bar,
                    #guianan_savanna_Bar, maranhao_babacu_Bar, zambezian_coastal_Bar, pernambuco_interior_Bar, zambeizan_and_mopane_Bar], loc='upper left', fontsize=31, ncol=5)

#Title
plt.title("(a)", fontsize=70, loc='left', fontweight='bold')


plt.show()

In [ ]:
#Conducting the calculation of the regions that are significant and insignificant compared to the Cerrado

#Defining the statistic for use below
def statistic(x, y):
    return np.median(x) - np.median(y)


#The Cerrado is 60704, so comparing all other regions to that, and going from there
temp_values = all_IAV_values[60704].values()
cerrado_values = []
for value in temp_values:
    cerrado_values.append(float(value))
print(f"The cerrado values: {cerrado_values}")
array1 = np.array(cerrado_values)
cerrado_median = np.median(array1)

#Counters to keep track of how many less than 0.05 and above 0.05 (this is the signifcance threshold)
count_sig = 0
count_insig = 0
for region in all_IAV_values:
    #Skipping the cerrado itself
    if region == 60704:
        continue
    #Making the values floats
    region_values = []
    for value in all_IAV_values[region].values():
        region_values.append(float(value))
    array2 = np.array(region_values)
    #Conducting the permutation tests
    result = permutation_test((array1, array2), statistic, permutation_type='samples', n_resamples = 262144, alternative='greater')
    #Printing the one-sided result of the permutation test
    print(f"True difference in median cerado - {region} = {cerrado_median - np.median(array2)}")
    
    #Counting if significant or insignificant
    if result.pvalue < 0.05:
        count_sig += 1
    else:
        count_insig += 1

print(f"The number of ecoregions the Cerrado is significnatly greater than: {count_sig} and the number it is not greater than: {count_insig}")

In [ ]:
#This block is concerned with getting the area-weighted contribution values for the top absolute contributing regions

#The area-weighted values for the absolute top contributing regions

#Cerrado 60704
cerrado_area_weighted_values = []
for model in area_weighted_IAV_values[60704]:
    cerrado_area_weighted_values.append(area_weighted_IAV_values[60704][model])
print(f"Cerrado Median Area-Weighted: {round(np.median(cerrado_area_weighted_values), 2)}")
print(f"Cerrado 25: {round(np.percentile(np.array(cerrado_area_weighted_values), 25), 2)}")
print(f"Cerrado 75: {round(np.percentile(np.array(cerrado_area_weighted_values), 75), 2)}")
#East Siberian 80601
east_siberian_area_weighted_values = []
for model in area_weighted_IAV_values[80601]:
    east_siberian_area_weighted_values.append(area_weighted_IAV_values[80601][model])
print(f"\nEast Siberian Median Area-Weighted: {round(np.median(east_siberian_area_weighted_values), 2)}")
print(f"East Siberian 25: {round(np.percentile(np.array(east_siberian_area_weighted_values), 25), 2)}")
print(f"East Siberian 75: {round(np.percentile(np.array(east_siberian_area_weighted_values), 75), 2)}")
#Miomobo 30704
miombo_area_weighted_values = []
for model in area_weighted_IAV_values[30704]:
    miombo_area_weighted_values.append(area_weighted_IAV_values[30704][model])
print(f"\nCentral Zambezian Miombo Median Area-Weighted: {round(np.median(miombo_area_weighted_values), 2)}")
print(f"Central Zambezian Miombo 25: {round(np.percentile(np.array(miombo_area_weighted_values), 25), 2)}")
print(f"Central Zambezian Miombo 75: {round(np.percentile(np.array(miombo_area_weighted_values), 75), 2)}")
#Caatinga 61304
caatinga_area_weighted_values = []
for model in area_weighted_IAV_values[61304]:
    caatinga_area_weighted_values.append(area_weighted_IAV_values[61304][model])
print(f"\nCaatinga Median Area-Weighted: {round(np.median(caatinga_area_weighted_values), 2)}")
print(f"Caatinga 25: {round(np.percentile(np.array(caatinga_area_weighted_values), 25), 2)}")
print(f"Caatinga 75: {round(np.percentile(np.array(caatinga_area_weighted_values), 75), 2)}")
#Mopane 30725
mopane_area_weighted_values = []
for model in area_weighted_IAV_values[30725]:
    mopane_area_weighted_values.append(area_weighted_IAV_values[30725][model])
print(f"\nZambezian and Mopane Woodlands Area-Weighted: {round(np.median(mopane_area_weighted_values), 2)}")
print(f"Zambezian and Mopane Woodlands 25: {round(np.percentile(np.array(mopane_area_weighted_values), 25), 2)}")
print(f"Zambezian and Mopane Woodlands 75: {round(np.percentile(np.array(mopane_area_weighted_values), 75), 2)}")
#South miombo 30719
south_miombo_area_weighted_values = []
for model in area_weighted_IAV_values[30719]:
    south_miombo_area_weighted_values.append(area_weighted_IAV_values[30719][model])
print(f"\nSouth Miombo Area-Weighted: {round(np.median(south_miombo_area_weighted_values), 2)}")
print(f"South Miombo 25: {round(np.percentile(np.array(south_miombo_area_weighted_values), 25), 2)}")
print(f"South Miombo 75: {round(np.percentile(np.array(south_miombo_area_weighted_values), 75), 2)}")
#Amazon Moist 60173
amazon_moist_area_weighted_values = []
for model in area_weighted_IAV_values[60173]:
    amazon_moist_area_weighted_values.append(area_weighted_IAV_values[60173][model])
print(f"\nUatuma-trombetas Area-Weighted: {round(np.median(amazon_moist_area_weighted_values), 2)}")
print(f"Uatuma-trombetas 25: {round(np.percentile(np.array(amazon_moist_area_weighted_values), 25), 2)}")
print(f"Uatuma-trombetas 75: {round(np.percentile(np.array(amazon_moist_area_weighted_values), 75), 2)}")
#Guianan Moist 60125
guianan_moist_area_weighted_values = []
for model in area_weighted_IAV_values[60125]:
    guianan_moist_area_weighted_values.append(area_weighted_IAV_values[60125][model])
print(f"\nGuianan Moist Area-Weighted: {round(np.median(guianan_moist_area_weighted_values), 2)}")
print(f"Guianan Moist 25: {round(np.percentile(np.array(guianan_moist_area_weighted_values), 25), 2)}")
print(f"Guianan Moist 75: {round(np.percentile(np.array(guianan_moist_area_weighted_values), 75), 2)}")
#West sudan 30722
west_sudan_area_weighted_values = []
for model in area_weighted_IAV_values[30722]:
    west_sudan_area_weighted_values.append(area_weighted_IAV_values[30722][model])
print(f"\nWest sudan Area-Weighted: {round(np.median(west_sudan_area_weighted_values), 2)}")
print(f"West sudan 25: {round(np.percentile(np.array(west_sudan_area_weighted_values), 25), 2)}")
print(f"West sudan 75: {round(np.percentile(np.array(west_sudan_area_weighted_values), 75), 2)}")
#Scandinavian 80608
scandinavian_area_weighted_values = []
for model in area_weighted_IAV_values[80608]:
    scandinavian_area_weighted_values.append(area_weighted_IAV_values[80608][model])
print(f"\nScandinavian Area-Weighted: {round(np.median(scandinavian_area_weighted_values), 2)}")
print(f"Scandinavian 25: {round(np.percentile(np.array(scandinavian_area_weighted_values ), 25), 2)}")
print(f"Scandinavian 75: {round(np.percentile(np.array(scandinavian_area_weighted_values ), 75), 2)}")





In [ ]:
#This block is concerned with getting the absolute contribution values for the top area-weighted contributing ecoregions

#Lists to add the values to
absolute_western_zambezian_grasslands = []
absolute_eastern_zimbabwe_montane = []
absolute_zambezian_baikiaea = []
absolute_southern_miombo = []
absolute_chao_phraya = []
absolute_guianan_savanna = []
absolute_maranhao_babacu = []
absolute_zambezian_coastal = []
absolute_pernambuco_interior = []
absolute_zambeizan_and_mopane = []

#Populating the values
for region in all_IAV_values:
    if int(region) in (30724, 31006, 30726, 30719, 40108, 60707, 60139, 30906, 60152, 30725):
    # For each model in each region
        for model in all_IAV_values[region]:
            if region == 30724:
                absolute_western_zambezian_grasslands.append(all_IAV_values[region][model]) 
            if region == 31006:
                absolute_eastern_zimbabwe_montane.append(all_IAV_values[region][model])
            if region == 30726:
                absolute_zambezian_baikiaea.append(all_IAV_values[region][model])
            if region == 30719:
                absolute_southern_miombo.append(all_IAV_values[region][model])
            if region == 40108:
                absolute_chao_phraya.append(all_IAV_values[region][model])
            if region == 60707:
                absolute_guianan_savanna.append(all_IAV_values[region][model])
            if region == 60139:
                absolute_maranhao_babacu.append(all_IAV_values[region][model])
            if region == 30906:
                absolute_zambezian_coastal.append(all_IAV_values[region][model])
            if region == 60152:
                absolute_pernambuco_interior.append(all_IAV_values[region][model])
            if region == 30725:
                absolute_zambeizan_and_mopane.append(all_IAV_values[region][model])
            
#Printing the median values and the IQR values
print(f"\nWestern Zambezian Grasslands Absolute: {round(np.median(absolute_western_zambezian_grasslands), 2)}")
print(f"Western Zambezian Grasslands Absolute 25: {round(np.percentile(np.array(absolute_western_zambezian_grasslands), 25), 2)}")
print(f"Western Zambezian Grasslands Absolute 75: {round(np.percentile(np.array(absolute_western_zambezian_grasslands), 75), 2)}")

print(f"\nEastern Zimbabwe Montane Absolute: {round(np.median(absolute_eastern_zimbabwe_montane), 2)}")
print(f"Eastern Zimbabwe Montane Absolute 25: {round(np.percentile(np.array(absolute_eastern_zimbabwe_montane), 25), 2)}")
print(f"Eastern Zimbabwe Montane Absolute 75: {round(np.percentile(np.array(absolute_eastern_zimbabwe_montane), 75), 2)}")

print(f"\nZambezian Baikiaea Absolute: {round(np.median(absolute_zambezian_baikiaea), 2)}")
print(f"Zambezian Baikiaea Absolute 25: {round(np.percentile(np.array(absolute_zambezian_baikiaea), 25), 2)}")
print(f"Zambezian Baikiaea Absolute 75: {round(np.percentile(np.array(absolute_zambezian_baikiaea), 75), 2)}")

print(f"\nSouthern Miombo Absolute: {round(np.median(absolute_southern_miombo), 2)}")
print(f"Southern Miombo Absolute 25: {round(np.percentile(np.array(absolute_southern_miombo), 25), 2)}")
print(f"Southern Miombo Absolute 75: {round(np.percentile(np.array(absolute_southern_miombo), 75), 2)}")

print(f"\nChao Phraya Absolute: {round(np.median(absolute_chao_phraya), 2)}")
print(f"Chao Phraya Absolute 25: {round(np.percentile(np.array(absolute_chao_phraya), 25), 2)}")
print(f"Chao Phraya Absolute 75: {round(np.percentile(np.array(absolute_chao_phraya), 75), 2)}")

print(f"\nGuianan Savanna Absolute: {round(np.median(absolute_guianan_savanna), 2)}")
print(f"Guianan Savanna Absolute 25: {round(np.percentile(np.array(absolute_guianan_savanna), 25), 2)}")
print(f"Guianan Savanna Absolute 75: {round(np.percentile(np.array(absolute_guianan_savanna), 75), 2)}")

print(f"\nMaranhao Babacu Absolute: {round(np.median(absolute_maranhao_babacu ), 2)}")
print(f"Maranhao Babacu Absolute 25: {round(np.percentile(np.array(absolute_maranhao_babacu), 25), 2)}")
print(f"Maranhao Babacu Absolute 75: {round(np.percentile(np.array(absolute_maranhao_babacu), 75), 2)}")

print(f"\nZambezian Coastal Absolute: {round(np.median(absolute_zambezian_coastal), 2)}")
print(f"Zambezian Coastal Absolute 25: {round(np.percentile(np.array(absolute_zambezian_coastal), 25), 2)}")
print(f"Zambezian Coastal Absolute 75: {round(np.percentile(np.array(absolute_zambezian_coastal), 75), 2)}")

print(f"\nPernambuco interior Absolute: {round(np.median(absolute_pernambuco_interior), 2)}")
print(f"Pernambuco interior Absolute 25: {round(np.percentile(np.array(absolute_pernambuco_interior), 25), 2)}")
print(f"Pernambuco interior Absolute 75: {round(np.percentile(np.array(absolute_pernambuco_interior), 75), 2)}")

print(f"\nZambezian and Mopane Absolute: {round(np.median(absolute_zambeizan_and_mopane ), 2)}")
print(f"Zambezian and Mopane Absolute 25: {round(np.percentile(np.array(absolute_zambeizan_and_mopane), 25), 2)}")
print(f"Zambezian and Mopane Absolute 75: {round(np.percentile(np.array(absolute_zambeizan_and_mopane), 75), 2)}")

